<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# Peer Review Assignment - Data Engineer - ETL


Estimated time needed: **20** minutes


## Objectives

In this final part you will:

- Run the ETL process
- Extract bank and market cap data from the JSON file `bank_market_cap.json`
- Transform the market cap currency using the exchange rate data
- Load the transformed data into a seperate CSV


For this lab, we are going to be using Python and several Python libraries. Some of these libraries might be installed in your lab environment or in SN Labs. Others may need to be installed by you. The cells below will install these libraries when executed.


In [1]:
#!mamba install pandas==1.3.3 -y
#!mamba install requests==2.26.0 -y

## Imports

Import any additional libraries you may need here.


In [2]:
import glob
import pandas as pd
from datetime import datetime

As the exchange rate fluctuates, we will download the same dataset to make marking simpler. This will be in the same format as the dataset you used in the last section  


In [3]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_2.json
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Final%20Assignment/exchange_rates.csv

--2023-10-13 13:17:59--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Transform%20Load/data/bank_market_cap_1.json
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2815 (2.7K) [application/json]
Saving to: ‘bank_market_cap_1.json’

bank_market_cap_1.j 100%[===================>]   2.75K  --.-KB/s    in 0s      

2023-10-13 13:18:00 (16.7 MB/s) - ‘bank_market_cap_1.json’ saved [2815/2815]

--2023-10-13 13:18:00--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0221EN-SkillsNetwork/labs/module%206/Lab%20-%20Extract%20Tra

## Extract


### JSON Extract Function

This function will extract JSON files.


In [19]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [26]:
columns=['Bank Name','Market Cap (US$ Billion)']

In [38]:
def extract():
    extracted_data = pd.DataFrame(columns=columns)
    for jsonfile in glob.glob("bank_market_cap.json"):
        extracted_data = extracted_data.append(extract_from_json(jsonfile), ignore_index=True)
    rate = exchange_rate()
    return extracted_data, rate


In [28]:
print(extract())

                                  Bank Name  Market Cap (US$ Billion)
0                            JPMorgan Chase                   390.934
1   Industrial and Commercial Bank of China                   345.214
2                           Bank of America                   325.331
3                               Wells Fargo                   308.013
4                   China Construction Bank                   257.399
..                                      ...                       ...
65                             Ping An Bank                    37.993
66                       Standard Chartered                    37.319
67                     United Overseas Bank                    35.128
68                                QNB Group                    33.560
69                              Bank Rakyat                    33.081

[70 rows x 2 columns]


In [33]:
def exchange_rate():
    df = pd.read_csv('exchange_rates_1.csv', index_col=0)
    exchange_rate = df.loc[df.index == 'GBP', 'Rate'].values[0]
    return exchange_rate


In [18]:
exchange_rate()

                 Rate
Currency             
AED          3.870864
AFN         79.795360
ALL        105.952770
AMD        423.867620
ANG          1.899235
...               ...
YER        263.784010
ZAR         19.887041
ZMK       9485.888279
ZMW         22.473756
ZWL        339.338248

[170 rows x 1 columns]
Exchange Rate for GBP: 0.864396


## Transform

Using <code>exchange_rate</code> and the `exchange_rates.csv` file find the exchange rate of USD to GBP. Write a transform function that

1. Changes the `Market Cap (US$ Billion)` column from USD to GBP
2. Rounds the Market Cap (US$ Billion)` column to 3 decimal places
3. Rename `Market Cap (US$ Billion)` to `Market Cap (GBP$ Billion)`


In [29]:
def transform(data, rate):
    data['Market Cap (US$ Billion)'] = round(data["Market Cap (US$ Billion)"] / rate , 2)
    data = data.rename(columns={"Market Cap (US$ Billion)": "Market Cap (GBP$ Billion)"})
    return data
    

## Load

Create a function that takes a dataframe and load it to a csv named `bank_market_cap_gbp.csv`. Make sure to set `index` to `False`.


In [30]:
def load(data_to_load):
    data_to_load.to_csv("bank_market_cap_gbp.csv", index=False)  

## Logging Function


Write the logging function <code>log</code> to log your data:


In [31]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile_result.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running the ETL Process


Log the process accordingly using the following <code>"ETL Job Started"</code> and <code>"Extract phase Started"</code>


In [32]:
# Write your code here
log("ETL Job Started")
log("Extract phase Started")

### Extract


<code>Question 2</code> Use the function <code>extract</code>, and print the first 5 rows, take a screen shot:


In [39]:
# Call the function here
data, rate = extract()
# Print the rows here
data.head()

,Bank Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


Log the data as <code>"Extract phase Ended"</code>


In [35]:
# Write your code here
log("Extract phase Ended")


### Transform


Log the following  <code>"Transform phase Started"</code>


In [36]:
# Write your code here
log("Transform phase Started")

<code>Question 3</code> Use the function <code>transform</code> and print the first 5 rows of the output, take a screen shot:


In [40]:
# Call the function here
transformed_data = transform(data, rate)
# Print the first 5 rows here
transformed_data.head()

,Bank Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,452.26
1,Industrial and Commercial Bank of China,399.37
2,Bank of America,376.37
3,Wells Fargo,356.33
4,China Construction Bank,297.78


Log your data <code>"Transform phase Ended"</code>


In [41]:
# Write your code here
log("Transform phase Ended")

### Load


Log the following `"Load phase Started"`.


In [42]:
# Write your code here
log("Load phase Started")

Call the load function


In [44]:
# Write your code here
load(transformed_data)

Log the following `"Load phase Ended"`.


In [45]:
# Write your code here
log("Load phase Ended")

## Authors


Ramesh Sannareddy, Joseph Santrcangelo and Azim Hirjani


### Other Contributors


Rav Ahuja


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By        | Change Description                 |
| ----------------- | ------- | ----------------- | ---------------------------------- |
| 2020-11-25        | 0.1     | Ramesh Sannareddy | Created initial version of the lab |


 Copyright © 2020 IBM Corporation. This notebook and its source code are released under the terms of the [MIT License](https://cognitiveclass.ai/mit-license?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkPY0221ENSkillsNetwork899-2023-01-01&cm_mmc=Email_Newsletter-_-Developer_Ed%2BTech-_-WW_WW-_-SkillsNetwork-Courses-IBM-DA0321EN-SkillsNetwork-21426264&cm_mmca1=000026UJ&cm_mmca2=10006555&cm_mmca3=M12345678&cvosrc=email.Newsletter.M12345678&cvo_campaign=000026UJ).
